In [7]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, BatchNormalization, Dropout
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.optimizers import Adam
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_absolute_error, r2_score

print('TensorFlow version:', tf.__version__)

TensorFlow version: 2.13.1


In [8]:
X_train = pd.read_csv('../data/processed/X_train.csv')
X_test  = pd.read_csv('../data/processed/X_test.csv')
y_train = pd.read_csv('../data/processed/y_train.csv').values.ravel()
y_test  = pd.read_csv('../data/processed/y_test.csv').values.ravel()

# Convert to float32
X_train = X_train.astype('float32')
X_test  = X_test.astype('float32')
y_train = y_train.astype('float32')
y_test  = y_test.astype('float32')

# ── Feature Scaling ──────────────────────────────────────────────────────────
# StandardScaler eliminates gradient instability caused by different feature
# magnitudes — the biggest single cause of val_loss spikes in the baseline.
scaler_X = StandardScaler()
X_train_scaled = scaler_X.fit_transform(X_train)   # fit ONLY on train data
X_test_scaled  = scaler_X.transform(X_test)

# Scale the target too so the network learns smaller numbers
scaler_y = StandardScaler()
y_train_scaled = scaler_y.fit_transform(y_train.reshape(-1, 1)).ravel()
y_test_scaled  = scaler_y.transform(y_test.reshape(-1, 1)).ravel()

print('Data loaded, converted, and scaled successfully.')
print('X_train shape:', X_train_scaled.shape)

Data loaded, converted, and scaled successfully.
X_train shape: (1106, 53)


In [9]:
# ── Build Improved Model ─────────────────────────────────────────────────────
# Improvements over baseline:
#   • Deeper network  : 256 → 128 → 64 → 32 → 1
#   • BatchNormalization: stabilises activations, speeds convergence
#   • Dropout(0.3)    : reduces overfitting on small dataset (~1 106 rows)

def build_improved_model(input_dim):
    model = Sequential([
        # Block 1
        Dense(256, activation='relu', input_shape=(input_dim,)),
        BatchNormalization(),
        Dropout(0.3),

        # Block 2
        Dense(128, activation='relu'),
        BatchNormalization(),
        Dropout(0.3),

        # Block 3
        Dense(64, activation='relu'),
        BatchNormalization(),
        Dropout(0.2),

        # Block 4
        Dense(32, activation='relu'),
        BatchNormalization(),

        # Output  (linear activation for regression)
        Dense(1)
    ])
    return model

model = build_improved_model(X_train_scaled.shape[1])
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_5 (Dense)             (None, 256)               13824     
                                                                 
 batch_normalization_4 (Bat  (None, 256)               1024      
 chNormalization)                                                
                                                                 
 dropout_3 (Dropout)         (None, 256)               0         
                                                                 
 dense_6 (Dense)             (None, 128)               32896     
                                                                 
 batch_normalization_5 (Bat  (None, 128)               512       
 chNormalization)                                                
                                                                 
 dropout_4 (Dropout)         (None, 128)              

In [10]:
# ── Compile Model ─────────────────────────────────────────────────────────────
model.compile(
    optimizer=Adam(learning_rate=0.001),
    loss='mean_squared_error',
    metrics=['mean_absolute_error']
)

In [11]:
# ── Callbacks ─────────────────────────────────────────────────────────────────
early_stop = EarlyStopping(
    monitor='val_loss',
    patience=20,                  # wait 20 epochs with no improvement
    restore_best_weights=True,    # roll back to best weights automatically
    verbose=1
)

reduce_lr = ReduceLROnPlateau(
    monitor='val_loss',
    factor=0.5,                   # halve LR when plateau is detected
    patience=8,
    min_lr=1e-6,
    verbose=1
)

# ── Train ─────────────────────────────────────────────────────────────────────
history = model.fit(
    X_train_scaled, y_train_scaled,
    epochs=300,                   # early stopping handles actual termination
    batch_size=32,
    validation_data=(X_test_scaled, y_test_scaled),
    callbacks=[early_stop, reduce_lr],
    verbose=1
)

Epoch 1/300
35/35 [==============================] - 1s 7ms/step - loss: 1.7471 - mean_absolute_error: 1.0094 - val_loss: 0.5942 - val_mean_absolute_error: 0.6130 - lr: 0.0010
Epoch 2/300
35/35 [==============================] - 0s 3ms/step - loss: 0.9348 - mean_absolute_error: 0.7541 - val_loss: 0.5372 - val_mean_absolute_error: 0.5832 - lr: 0.0010
Epoch 3/300
35/35 [==============================] - 0s 3ms/step - loss: 0.6419 - mean_absolute_error: 0.6333 - val_loss: 0.4746 - val_mean_absolute_error: 0.5541 - lr: 0.0010
Epoch 4/300
35/35 [==============================] - 0s 3ms/step - loss: 0.5617 - mean_absolute_error: 0.5883 - val_loss: 0.3630 - val_mean_absolute_error: 0.4831 - lr: 0.0010
Epoch 5/300
35/35 [==============================] - 0s 3ms/step - loss: 0.5100 - mean_absolute_error: 0.5570 - val_loss: 0.2915 - val_mean_absolute_error: 0.4251 - lr: 0.0010
Epoch 6/300
35/35 [==============================] - 0s 3ms/step - loss: 0.4510 - mean_absolute_error: 0.5223 - val_loss

In [12]:
# ── Evaluate Model ────────────────────────────────────────────────────────────
# Predict on scaled test data, then invert scaling for real-world metrics
y_pred_scaled = model.predict(X_test_scaled).ravel()
y_pred = scaler_y.inverse_transform(y_pred_scaled.reshape(-1, 1)).ravel()

mae = mean_absolute_error(y_test, y_pred)
r2  = r2_score(y_test, y_pred)

print('=' * 40)
print(f'  Baseline MAE : ~0.6639')
print(f'  Improved MAE : {mae:.4f}')
print(f'  R² Score     : {r2:.4f}  (1.0 = perfect)')
print('=' * 40)

if mae < 0.6639:
    print(f' MAE improved by {((0.6639 - mae) / 0.6639 * 100):.1f}%')
else:
    print(' MAE did not improve — consider re-running (random seed variation)')

7/7 [==============================] - 0s 968us/step
  Baseline MAE : ~0.6639
  Improved MAE : 0.1751
  R² Score     : 0.8544  (1.0 = perfect)
 MAE improved by 73.6%
